In [34]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve,least_squares,root
from scipy import optimize
from scipy.integrate import solve_ivp
import pandas as pd

In [35]:
A = np.array([[5/36, 2/9 - np.sqrt(15)/15, 5/36 - np.sqrt(15)/30],[5/36 + np.sqrt(15)/24, 2/9, 5/36 - np.sqrt(15)/24],[5/36 + np.sqrt(15)/30, 2/9 + np.sqrt(15)/15, 5/36]],dtype=np.float64)
c= np.array([1/2 - np.sqrt(15)/10, 1/2,1/2 + np.sqrt(15)/10],dtype=np.float64)
b= np.array([5/18, 4/9,5/18],dtype=np.float64)
b_tilda=np.array([-5/6,8/3,-5/6],dtype=np.float64)
pd.set_option('display.precision', 15)

In [36]:
def Thomas_Fermi(x,z):
    u,v=z
    return [v,np.power(u,1.5)/np.sqrt(x)]
def TF1(x,u,v):
     return v
def TF2(x,u,v):
    return np.power(u,1.5)/np.sqrt(x)

In [37]:
def Find_K(K,x_k,u_k,v_k,h):
    k1_u,k2_u,k3_u,k1_v,k2_v,k3_v=K
    eq1=k1_u-TF1(x_k+c[0]*h,u_k+h*(A[0,0]*k1_u+A[0,1]*k2_u+A[0,2]*k3_u),v_k+h*(A[0,0]*k1_v+A[0,1]*k2_v+A[0,2]*k3_v))
    eq2=k2_u-TF1(x_k+c[1]*h,u_k+h*(A[1,0]*k1_u+A[1,1]*k2_u+A[1,2]*k3_u),v_k+h*(A[1,0]*k1_v+A[1,1]*k2_v+A[1,2]*k3_v))
    eq3=k3_u-TF1(x_k+c[2]*h,u_k+h*(A[2,0]*k1_u+A[2,1]*k2_u+A[2,2]*k3_u),v_k+h*(A[2,0]*k1_v+A[2,1]*k2_v+A[2,2]*k3_v))
    eq4=k1_v-TF2(x_k+c[0]*h,u_k+h*(A[0,0]*k1_u+A[0,1]*k2_u+A[0,2]*k3_u),v_k+h*(A[0,0]*k1_v+A[0,1]*k2_v+A[0,2]*k3_v))
    eq5=k2_v-TF2(x_k+c[1]*h,u_k+h*(A[1,0]*k1_u+A[1,1]*k2_u+A[1,2]*k3_u),v_k+h*(A[1,0]*k1_v+A[1,1]*k2_v+A[1,2]*k3_v))
    eq6=k3_v-TF2(x_k+c[2]*h,u_k+h*(A[2,0]*k1_u+A[2,1]*k2_u+A[2,2]*k3_u),v_k+h*(A[2,0]*k1_v+A[2,1]*k2_v+A[2,2]*k3_v))
    return [eq1,eq2,eq3,eq4,eq5,eq6]

In [38]:

def Evaluate_K(h,K,u,v,point):
    sol=root(Find_K,K,args=(point,u,v,h))
    k1u,k2u,k3u,k1v,k2v,k3v=sol.x
    K=[k1u,k2u,k3u,k1v,k2v,k3v]
    return K

In [39]:
def Evaluate(K,u,v,h):
    k1u,k2u,k3u,k1v,k2v,k3v=K
    u_next=u+h*(b[0]*k1u+b[1]*k2u+b[2]*k3u)
    v_next=v+h*(b[0]*k1v+b[1]*k2v+b[2]*k3v) 
    u_tilda=u+h*(b_tilda[0]*k1u+b_tilda[1]*k2u+b_tilda[2]*k3u)
    v_tilda=v+h*(b_tilda[0]*k1v+b_tilda[1]*k2v+b_tilda[2]*k3v)
    return u_next,v_next,u_tilda,v_tilda

In [40]:
def Step_update(u_next,v_next,u_tilda,v_tilda,h,x_old,K,sol,dist,j,X,epsilon_0):
    step_error=np.sqrt(np.power(u_next-u_tilda,2)+np.power(v_next-v_tilda,2))
    x_curr=x_old
    n_iter=0
    n_iter1=0
    while step_error>epsilon_0 and n_iter<100:
        h=beta*h*np.power((epsilon_0/step_error),1/7)
        h=max(h,1e-32)
        x_try=x_old+h
        K=Evaluate_K(h,K,sol[j-1][0],sol[j-1][1],dist[j-1])
        u_next,v_next,u_tilda,v_tilda=Evaluate(K,sol[j-1][0],sol[j-1][1],h)
        step_error=np.sqrt(np.power(u_next-u_tilda,2)+np.power(v_next-v_tilda,2))
        n_iter+= 1
    if x_old+h>X:
        h=X-x_old
        K=Evaluate_K(h,K,sol[j-1][0],sol[j-1][1],dist[j-1])
        u_next,v_next,u_tilda,v_tilda=Evaluate(K,sol[j-1][0],sol[j-1][1],h)
        x_curr=X
    else:
        x_curr=x_old+h 
    if u_next>=0:
        h_new=beta_1*h*np.power((epsilon_0/step_error),1/6)
        dist.append(x_curr)
        sol.append([u_next, v_next])
        return h_new,K
    else:
        while n_iter1 < 4:
            h = h / 10
            K = Evaluate_K(h, K, sol[j-1][0], sol[j-1][1], dist[j-1])
            u_next, v_next, u_tilda, v_tilda = Evaluate(K, sol[j-1][0], sol[j-1][1], h)
            if u_next >= 0:
                break
            n_iter1 += 1
        else: 
            return 0,K
    
        x_curr = x_old + h
        h_new = beta * h * np.power((epsilon_0 / step_error), 1/6)
        dist.append(x_curr)
        sol.append([u_next, v_next])
        return h_new,K

In [41]:
def RK26_implicit(h_init,x_0,X,u_0,v_0,epsilon_0):
    Solution=[]
    Dist=[]
    distance=[x_0]
    sol=[[u_0,v_0]]
    h=h_init
    K=np.zeros(6)
    h_curr=h
    j=1
    x_curr=x_0
    epsilon_0=epsilon_0
    while distance[j-1]<X:
        x_curr=distance[j-1]
        K=Evaluate_K(h_curr,K,sol[j-1][0],sol[j-1][1],distance[j-1])
        u_next,v_next,u_tilda,v_tilda=Evaluate(K,sol[j-1][0],sol[j-1][1],h_curr)
        h_curr,K=Step_update(u_next,v_next,u_tilda,v_tilda,h_curr,x_curr,K,sol,distance,j,X,epsilon_0)
        if h_curr!=0:
            j+=1
        else: break
    sol=np.array(sol)
    distance=np.array(distance)
    return sol,distance

In [ ]:
x0=np.power(10.0,-235)
v_=-1.5880710226113753127186845094239501093
epsilon_0=np.power(10.0,-13.5)
beta=0.9
beta_1=0.9
X_list=[4,5,15,25,50,100,200,300,400,500,1000]

In [43]:
method=['Radau','RK45','RK23','DOP853','LSODA']
df=pd.DataFrame(columns=['method','last node','value at the end','steps'])

In [44]:
for X in X_list:
    sol,d=RK26_implicit(1/1000,0,X,1,v_,epsilon_0)
    df.loc[len(df)]=['RK26_implicit',d[-1],sol[-1,0],np.shape(d)[0]]

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15112\1426277869.py:22: RuntimeWarning: divide by zero encountered in scalar divide
  h_new=beta_1*h*np.power((epsilon_0/step_error),1/6)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_15112\4184207430.py:7: RuntimeWarning: invalid value encountered in scalar divide
  return np.power(u,1.5)/np.sqrt(x)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_15112\1426277869.py:2: RuntimeWarning: invalid value encountered in scalar subtract
  step_error=np.sqrt(np.power(u_next-u_tilda,2)+np.power(v_next-v_tilda,2))
C:\Users\Administrator\AppData\Local\Temp\ipykernel_15112\4184207430.py:7: RuntimeWarning: invalid value encountered in power
  return np.power(u,1.5)/np.sqrt(x)


In [45]:

for metoda in method:
    for X in X_list:
        solve3=solve_ivp(Thomas_Fermi,[x0,X],[1,v_],method=metoda,rtol=2.220446049250313e-14,atol=2.220446049250313e-14)
        df.loc[len(df)]=[metoda,solve3.t[-1],solve3.y[0,-1],np.shape(solve3.t)[0]]
  

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15112\4184207430.py:3: RuntimeWarning: invalid value encountered in power
  return [v,np.power(u,1.5)/np.sqrt(x)]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_15112\4184207430.py:3: RuntimeWarning: invalid value encountered in power
  return [v,np.power(u,1.5)/np.sqrt(x)]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_15112\4184207430.py:3: RuntimeWarning: invalid value encountered in power
  return [v,np.power(u,1.5)/np.sqrt(x)]
c:\Users\Administrator\AppData\Local\Programs\Python\Python313\Lib\site-packages\scipy\integrate\_ivp\lsoda.py:161: UserWarning: lsoda: Excess accuracy requested (tolerances too small).
  solver._y, solver.t = integrator.run(


In [46]:
df

,method,last node,value at the end,steps
0,RK26_implicit,4.000000000000000,1.084042569190990e-01,149771
1,RK26_implicit,5.000000000000000,7.880777925169809e-02,154362
2,RK26_implicit,15.000000000000000,1.080535876455029e-02,173098
3,RK26_implicit,25.000000000000000,3.473754471795099e-03,179122
4,RK26_implicit,50.000000000000000,6.322556388767699e-04,184729
...,...,...,...,...
61,LSODA,200.000000000000000,1.985649677973538e-05,1990
62,LSODA,300.000000000000000,4.612073691747066e-05,2015
63,LSODA,400.000000000000000,4.288840337094542e-04,2054
64,LSODA,500.000000000000000,5.532937703213914e-02,2190


In [47]:
End=df[len(X_list)-1:len(df):len(X_list)]
End


,method,last node,value at the end,steps
10,RK26_implicit,790.872491781784902,9.712332438494954e+01,595633
21,Radau,321.641618878859504,3.793672249628211e-20,10526
32,RK45,333.795969373626860,6.702559698944907e+42,20339
43,RK23,411.620499055685400,1.808520390326839e+35,1323063
54,DOP853,456.142208167189892,2.173337314548002e+47,2101
65,LSODA,544.010376827680034,4.439486778105007e+13,3365
